Importing Libraries

In [3]:
#libraries for building the autoencoder
from keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose
from keras.models import Model
from keras.preprocessing import image
from keras.optimizers import Adam
import tensorflow as tf
import optuna
import numpy as np
import os

#metric libraries
import time
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from skimage.color import rgb2gray


c:\Users\GOKAY\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Accessing The Image Directories and Looking For Images

In [4]:
#access image directories
train_dir = 'C:/Users/GOKAY/Desktop/imagenette4/train/'
val_dir = 'C:/Users/GOKAY/Desktop/imagenette4/val/'
test_dir = 'C:/Users/GOKAY/Desktop/imagenette4/test/'

#train set
train = []
for filename in os.listdir(train_dir):
    if filename.endswith(".JPEG"):
        img = image.load_img(train_dir+filename, target_size=(160, 160))
        train.append(image.img_to_array(img))
train = np.array(train)

#validation set
val = []
for filename in os.listdir(val_dir):
    if filename.endswith(".JPEG"):
        img = image.load_img(val_dir+filename, target_size=(160, 160))
        val.append(image.img_to_array(img))
val = np.array(val)

#test set
test = []
for filename in os.listdir(test_dir):
    if filename.endswith(".JPEG"):
        img = image.load_img(test_dir+filename, target_size=(160, 160))
        test.append(image.img_to_array(img))
test = np.array(test)

In [5]:
print("train", train.shape)
print("val", val.shape)
print("test", test.shape)

train (9469, 160, 160, 3)
val (1961, 160, 160, 3)
test (1964, 160, 160, 3)


Defining a Function For Model Creation (Which Is Changed For Each Architecture Manually)

In [7]:
def create_model(trial):
    #hypermeters to be found
    num_filters1 = trial.suggest_categorical('num_filters1', [32])
    num_filters2 = trial.suggest_categorical('num_filters2', [32,64])
    num_filters3 = trial.suggest_categorical('num_filters3', [32, 64, 128])
    activation = trial.suggest_categorical('activation', ['relu','leaky_relu'])
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-4, 1e-2)

    #encoder
    input_layer = Input(shape=(160,160,3), name='INPUT')
    #first layer
    x = Conv2D(num_filters1, (3, 3), activation=activation, padding='same')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    #second layer
    x = Conv2D(num_filters2, (3, 3), activation=activation, padding='same' )(x)
    x = MaxPooling2D((2, 2))(x)
    #third layer
    x = Conv2D(num_filters3, (3, 3), activation=activation, padding='same')(x)

    latent_layer = MaxPooling2D((2, 2), name='CODE')(x)

    #decoder
    #fourth layer
    x = Conv2DTranspose(num_filters3, (3, 3), activation=activation, padding='same')(latent_layer)
    x = UpSampling2D((2, 2))(x)
    #fifth layer
    x = Conv2DTranspose(num_filters2, (3, 3), activation=activation, padding='same')(x)
    x = UpSampling2D((2, 2))(x)
    #sixth layer
    x = Conv2DTranspose(num_filters1, (3, 3), activation=activation, padding='same')(x)
    x = UpSampling2D((2, 2))(x)

    output_layer = Conv2D(3, (3, 3), padding='same', name="OUTPUT")(x)
    
    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(optimizer=Adam(learning_rate=learning_rate), loss='mse')
    #autoencoder.summary()
    return autoencoder

Defining a Objective Function For Optuna

In [8]:
def objective(trial):
    #creating the model and setting up the parameters for testing.
    model = create_model(trial)
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
    start_time = time.time()
    history = model.fit(train, train, epochs=15, batch_size=32, validation_data=(val, val), callbacks=[early_stopping], verbose=1)
    time_taken = time.time()-start_time

    #prediction on val set
    val_pred = model.predict(val)
    ssim_scores = []
    psnr_scores = []

    for i in range(len(val)):
        #creating a new set of the same images which turned into gray scale for SSIM test. RGB images are problematic.
        val_gray = rgb2gray(val[i])
        val_pred_gray = rgb2gray(val_pred[i])

        #holding the scores of metrics
        ssim_score = ssim(val_gray, val_pred_gray, data_range=val_pred_gray.max() - val_pred_gray.min())
        psnr_score = psnr(val[i], val_pred[i], data_range=val_pred[i].max() - val_pred[i].min())
        ssim_scores.append(ssim_score)
        psnr_scores.append(psnr_score)
    
    #mean values of metrics and combined score. Optuna finds the minimum value. Therefore, combined score's sign is changed to minus in order to get the actual highest value.
    mean_ssim = np.mean(ssim_scores)
    mean_psnr = np.mean(psnr_scores)
    combined_score = -1 * (mean_ssim + mean_psnr)

    #attributes are set to receive them later on.
    val_loss = min(history.history['val_loss'])
    epochs_taken = len(history.history['val_loss'])
    trial.set_user_attr('epochs', epochs_taken)
    trial.set_user_attr('SSIM', mean_ssim)
    trial.set_user_attr('PSNR', mean_psnr)
    trial.set_user_attr('combined_score', combined_score)
    trial.set_user_attr('final_val_loss', val_loss)
    trial.set_user_attr("time_taken", time_taken)

    #combined score is returned, optuna watches for the returned variable in the objective function
    return combined_score

Studying the Hypermeters

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=20)

Finding The Best Trial and Its Hypermeters, According to the Combined Score

In [ ]:
best_trial = study.best_trial
print('Best hyperparameters for combined score:', best_trial.params)
print("Best SSIM:", best_trial.user_attrs['SSIM'])
print("Best PSNR:", best_trial.user_attrs['PSNR'])
print("Best Combined Score:",best_trial.user_attrs['combined_score'])
print('Number of epochs:', best_trial.user_attrs['epochs'])
print('Final validation loss:', best_trial.user_attrs['final_val_loss'])
print('Time taken for the best result: ', best_trial.user_attrs['time_taken'])